In [ ]:
#This code detects objects using Faster RCNN
#You may need to install the following libraries

In [ ]:
#pip install torch torchvision

In [ ]:
import torch
import torchvision.transforms.functional as F
import torchvision.models.detection as detection
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [ ]:
#We will use the pre-trained FasterRCNN_ResNet50_FPN model. This model is trained with the COCO dataset
#Let us see the COCO dataset classes. These classes will be printed in the image
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights

# Load the metadata for the pre-trained model
weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
COCO_CLASSES = weights.meta["categories"]  # Fetch the class names dynamically

print(COCO_CLASSES) 

In [ ]:
# Step 1: Load the pre-trained Faster R-CNN model
model = detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Step 2: Load the image
image_path = "195.jpg"
image = Image.open(image_path)
# Convert the image to a tensor in the C (channels), H (height), and W (weight)
image_tensor = F.to_tensor(image) #Converts the input image (loaded using PIL) into a tensor format required by PyTorch models.


# Step 3: Perform object detection
with torch.no_grad():
    predictions = model([image_tensor])

# Step 4: Extract predictions
predicted_boxes = predictions[0]['boxes']  # Bounding boxes
predicted_scores = predictions[0]['scores']  # Confidence scores
predicted_labels = predictions[0]['labels']  # Class indices

#Filter predictions by confidence threshold. Let us assume that we are only interested in those prediction which are above 0.5
confidence_threshold = 0.5
filtered_boxes = []
filtered_scores = []
filtered_labels = []

for box, score, label in zip(predicted_boxes, predicted_scores, predicted_labels):
    if score >= confidence_threshold:
        filtered_boxes.append(box)
        filtered_scores.append(score)
        filtered_labels.append(label)

# Step 5: Annotate the image with bounding boxes and class names
fig, ax = plt.subplots(1, figsize=(12, 8), dpi=72)  # Set the figure size and resolution
ax.imshow(image)

for box, score, label in zip(filtered_boxes, filtered_scores, filtered_labels):
    x_min, y_min, x_max, y_max = box.tolist()
    class_name = COCO_CLASSES[label.item()]  # Map label to class name

    # Draw the bounding box
    rect = patches.Rectangle(
        (x_min, y_min),
        x_max - x_min,
        y_max - y_min,
        linewidth=2,
        edgecolor='red',
        facecolor='none'
    )
    ax.add_patch(rect)

    # Add the class name and score as text
    ax.text(
        x_min,
        y_min - 5,
        f"{class_name}: {score:.2f}",
        color='white',
        fontsize=10,
        bbox=dict(facecolor='red', alpha=0.5)
    )

plt.axis("off")  # Hide axes for better visualization
plt.show()